# Population map file cropping China

In [4]:
import geopandas as gpd

# 定义文件路径
geojson_file = 'counties.geojson'

# 读取 GeoJSON 文件
gdf = gpd.read_file(geojson_file,engine='pyogrio')

# 打印 GeoDataFrame 的 CRS
print(gdf.crs)

EPSG:4326


In [ ]:
import os
import glob
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from rasterio.errors import RasterioIOError

# Input and output path settings (adjust according to actual paths)
input_tif_directory = r'全球人口1km'
output_tif_directory = r'中国裁剪后人口1km全部'
geojson_path = r'中国_市.geojson'

# Create output directory if it does not exist
os.makedirs(output_tif_directory, exist_ok=True)

# Load the China municipal GeoJSON file and fix invalid geometries
print("Loading GeoJSON file...")
cities_gdf = gpd.read_file(geojson_path, engine='pyogrio')

# Ensure geometries are valid
if not cities_gdf.geometry.is_valid.all():
    print("Warning: Invalid geometries found in GeoDataFrame, attempting to fix...")
    cities_gdf["geometry"] = cities_gdf.geometry.buffer(0)

# Remove empty geometries
cities_gdf = cities_gdf[~cities_gdf.geometry.is_empty]
if cities_gdf.empty:
    print("Error: No valid geometries after repair. Please check the input data!")
    exit()

# Check and standardize coordinate system (convert EPSG:4490 to EPSG:4326 if needed)
if cities_gdf.crs.to_epsg() != 4326:
    print("Converting GeoJSON CRS to EPSG:4326...")
    cities_gdf = cities_gdf.to_crs(epsg=4326)

# Get list of all population TIF files
print("Retrieving list of TIF files...")
population_tifs = glob.glob(os.path.join(input_tif_directory, 'global_*_2020_1km.tif'))

if not population_tifs:
    print("No matching TIF files found. Please check the input directory.")
    exit()

# Print GeoJSON and TIF file bounds for debugging
print("GeoJSON bounds:", cities_gdf.total_bounds)


def clip_raster(tif_path, cities_gdf):
    """
    Clip a single TIF file using China city boundaries and save to the specified location.
    """
    output_filename = os.path.basename(tif_path).replace('.tif', '_clipped.tif')
    output_path = os.path.join(output_tif_directory, output_filename)
    
    try:
        print(f"Processing file: {tif_path}")
        with rasterio.open(tif_path) as src:
            # Check if the TIF file CRS is EPSG:4326
            if src.crs.to_epsg() != 4326:
                raise ValueError(f"File {tif_path} does not use EPSG:4326. Cannot process.")

            # Check if TIF and GeoJSON bounds intersect
            tif_bounds = src.bounds
            geo_bounds = cities_gdf.total_bounds
            if (tif_bounds[2] < geo_bounds[0] or  # TIF max longitude < GeoJSON min longitude
                tif_bounds[0] > geo_bounds[2] or  # TIF min longitude > GeoJSON max longitude
                tif_bounds[3] < geo_bounds[1] or  # TIF max latitude < GeoJSON min latitude
                tif_bounds[1] > geo_bounds[3]):   # TIF min latitude > GeoJSON max latitude
                print(f"Warning: File {tif_path} has no spatial overlap with GeoJSON. Skipping.")
                return

            # Clip the raster
            out_image, out_transform = mask(src, cities_gdf.geometry, crop=True)

            # Check if the clipped result is empty
            if out_image.size == 0:
                print(f"Warning: Clipping result for {tif_path} is empty. Skipping save.")
                return

            out_meta = src.meta.copy()
            
            # Update metadata to reflect the new extent
            out_meta.update({
                "driver": "GTiff",
                "height": out_image.shape[1],
                "width": out_image.shape[2],
                "transform": out_transform
            })
            
            # Write the clipped TIF file
            with rasterio.open(output_path, "w", **out_meta) as dest:
                dest.write(out_image)
                
            print(f"Successfully clipped and saved: {output_path}")
    except RasterioIOError as rio_err:
        print(f"Cannot open file {tif_path}: {rio_err}")
    except ValueError as ve:
        print(f"Problem with file {tif_path}: {ve}")
    except Exception as e:
        print(f"Error during clipping {tif_path}: {e}")


# Process each TIF file sequentially
print("Starting to clip TIF files...")
for tif in population_tifs:
    clip_raster(tif, cities_gdf)

print("All files have been clipped successfully!")

#  Multiband synthetic population in China

In [ ]:
import os
import glob
import rasterio
from rasterio.enums import Resampling

# Input and output path settings (adjust according to actual paths)
clipped_tif_directory = r'中国裁剪后人口1km全部'
output_multiband_tif_path = r'merged_population_data_chinanew.tif'

# Get all clipped population TIF files
clipped_population_tifs = glob.glob(os.path.join(clipped_tif_directory, '*_clipped.tif'))

# Ensure at least one file is available to create a multiband TIF
if not clipped_population_tifs:
    print("No clipped TIF files found.")
else:
    # Use the first file to get metadata template
    with rasterio.open(clipped_population_tifs[0]) as src:
        meta = src.meta.copy()
        meta.update(count=len(clipped_population_tifs))

    # Prepare list for band descriptions
    band_descriptions = []

    # Open a new file to write multiband data
    with rasterio.open(output_multiband_tif_path, 'w', **meta) as dst:
        for band_index, tif in enumerate(clipped_population_tifs, start=1):
            try:
                # Generate band description from filename
                filename = os.path.basename(tif)
                # Assume filename format: global_[f|m]_[age]_[year]_1km_clipped.tif or similar
                parts = filename.split('_')
                if len(parts) >= 4 and parts[-1].startswith('clipped'):
                    band_name = f"{parts[1]}_{parts[2]}"  # e.g., f_20 or m_75
                else:
                    band_name = f"band_{band_index}"  # default name

                print(f"Processing file {tif} as band {band_index} ({band_name})")

                # Write single-band data into the multiband file
                with rasterio.open(tif) as src:
                    data = src.read(1)
                    dst.write(data, band_index)

                # Add band description
                band_descriptions.append((band_index, band_name))
            except Exception as e:
                print(f"Error processing file {tif}: {e}")

        # Write band descriptions as tags in the multiband TIF file
        dst.update_tags(ns='rio_band_descriptions', **{str(i): desc for i, desc in band_descriptions})

    print(f"All files have been successfully merged into multiband TIF: {output_multiband_tif_path}")
    print("Band descriptions:")
    for i, desc in band_descriptions:
        print(f"Band {i}: {desc}")


#  Population map file cropping USA

In [ ]:
import os
import glob
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from rasterio.errors import RasterioIOError

# Input and output path settings (adjust according to actual paths)
input_tif_directory = r'全球人口1km'
output_tif_directory = r'美国裁剪后人口1km全部'
geojson_path = r'counties.geojson'

# Create output directory if it does not exist
os.makedirs(output_tif_directory, exist_ok=True)

# Load US county-level GeoJSON file and fix invalid geometries
print("Loading GeoJSON file...")
counties_gdf = gpd.read_file(geojson_path, engine='pyogrio')

# Ensure all geometries are valid
if not counties_gdf.geometry.is_valid.all():
    print("Warning: Invalid geometries found in GeoDataFrame, attempting to repair...")
    counties_gdf["geometry"] = counties_gdf.geometry.buffer(0)

# Remove empty geometries
counties_gdf = counties_gdf[~counties_gdf.geometry.is_empty]
if counties_gdf.empty:
    print("Error: No valid geometries after repair. Please check the data!")
    exit()

# Check if CRS is EPSG:4326
if counties_gdf.crs.to_epsg() != 4326:
    print("Warning: GeoJSON CRS is not EPSG:4326. Reprojecting...")
    counties_gdf = counties_gdf.to_crs(epsg=4326)

# Get list of all population TIF files
print("Retrieving list of TIF files...")
population_tifs = glob.glob(os.path.join(input_tif_directory, 'global_*_2020_1km.tif'))

if not population_tifs:
    print("No matching TIF files found. Please check the input directory.")
    exit()

# Print GeoJSON and TIF file bounds for debugging
print("GeoJSON bounds:", counties_gdf.total_bounds)


def clip_raster(tif_path, counties_gdf):
    """
    Clip a single TIF file using US county boundaries and save to the specified location.
    """
    output_filename = os.path.basename(tif_path).replace('.tif', '_clipped_usa.tif')
    output_path = os.path.join(output_tif_directory, output_filename)
    
    try:
        print(f"Processing file: {tif_path}")
        with rasterio.open(tif_path) as src:
            # Check if the TIF file uses EPSG:4326
            if src.crs.to_epsg() != 4326:
                raise ValueError(f"File {tif_path} does not use EPSG:4326. Cannot process.")

            # Check if TIF and GeoJSON bounds intersect
            tif_bounds = src.bounds
            geo_bounds = counties_gdf.total_bounds
            if (tif_bounds[2] < geo_bounds[0] or  # TIF max longitude < GeoJSON min longitude
                tif_bounds[0] > geo_bounds[2] or  # TIF min longitude > GeoJSON max longitude
                tif_bounds[3] < geo_bounds[1] or  # TIF max latitude < GeoJSON min latitude
                tif_bounds[1] > geo_bounds[3]):   # TIF min latitude > GeoJSON max latitude
                print(f"Warning: File {tif_path} has no spatial overlap with GeoJSON. Skipping.")
                return

            # Clip the raster
            out_image, out_transform = mask(src, counties_gdf.geometry, crop=True)

            # Check if the clipped result is empty
            if out_image.size == 0:
                print(f"Warning: Clipping result for {tif_path} is empty. Skipping save.")
                return

            out_meta = src.meta.copy()
            
            # Update metadata to reflect the new extent
            out_meta.update({
                "driver": "GTiff",
                "height": out_image.shape[1],
                "width": out_image.shape[2],
                "transform": out_transform
            })
            
            # Write the clipped TIF file
            with rasterio.open(output_path, "w", **out_meta) as dest:
                dest.write(out_image)
                
            print(f"Successfully clipped and saved: {output_path}")
    except RasterioIOError as rio_err:
        print(f"Cannot open file {tif_path}: {rio_err}")
    except ValueError as ve:
        print(f"Problem with file {tif_path}: {ve}")
    except Exception as e:
        print(f"Error during clipping {tif_path}: {e}")


# Process each TIF file sequentially
print("Starting to clip TIF files...")
for tif in population_tifs:
    clip_raster(tif, counties_gdf)

print("All files have been clipped successfully!")

#  Multiband synthetic population in USA

In [ ]:
import os
import glob
import rasterio
from rasterio.enums import Resampling

# Input and output path settings (adjust according to actual paths)
clipped_tif_directory = r'美国裁剪后人口1km全部'
output_multiband_tif_path = r'merged_population_data_USAnew.tif'

# Get all clipped population TIF files
clipped_population_tifs = glob.glob(os.path.join(clipped_tif_directory, '*_clipped_usa.tif'))

# Ensure at least one file is available to create a multiband TIF
if not clipped_population_tifs:
    print("No clipped TIF files found.")
else:
    # Use the first file to get metadata template
    with rasterio.open(clipped_population_tifs[0]) as src:
        meta = src.meta.copy()
        meta.update(count=len(clipped_population_tifs))

    # Prepare list for band descriptions
    band_descriptions = []

    # Open a new file to write multiband data
    with rasterio.open(output_multiband_tif_path, 'w', **meta) as dst:
        for band_index, tif in enumerate(clipped_population_tifs, start=1):
            try:
                # Generate band description from filename
                filename = os.path.basename(tif)
                # Assume filename format: global_[f|m]_[age]_[year]_1km_clipped_usa.tif or similar
                parts = filename.split('_')
                if len(parts) >= 4 and parts[-1].startswith('clipped'):
                    band_name = f"{parts[1]}_{parts[2]}"  # e.g., f_20 or m_75
                else:
                    band_name = f"band_{band_index}"  # default name

                print(f"Processing file {tif} as band {band_index} ({band_name})")

                # Write single-band data into the multiband file
                with rasterio.open(tif) as src:
                    data = src.read(1)
                    dst.write(data, band_index)

                # Add band description
                band_descriptions.append((band_index, band_name))
            except Exception as e:
                print(f"Error processing file {tif}: {e}")

        # Write band descriptions as tags in the multiband TIF file
        dst.update_tags(ns='rio_band_descriptions', **{str(i): desc for i, desc in band_descriptions})

    print(f"All files have been successfully merged into multiband TIF: {output_multiband_tif_path}")
    print("Band descriptions:")
    for i, desc in band_descriptions:
        print(f"Band {i}: {desc}")

#   Population map file cropping Europe

In [ ]:
import os
import glob
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from rasterio.errors import RasterioIOError

# Input and output path settings (adjust according to actual paths)
input_tif_directory = r'全球人口1km'
output_tif_directory = r'欧洲裁剪后人口1km全部'
geojson_path = r'NUTS_RG_20M_2021_4326.geojson'

# Create output directory if it does not exist
os.makedirs(output_tif_directory, exist_ok=True)

# Load European NUTS regions GeoJSON file and fix invalid geometries
print("Loading GeoJSON file...")
counties_gdf = gpd.read_file(geojson_path, engine='pyogrio')

# Ensure all geometries are valid
if not counties_gdf.geometry.is_valid.all():
    print("Warning: Invalid geometries found in GeoDataFrame, attempting to repair...")
    counties_gdf["geometry"] = counties_gdf.geometry.buffer(0)

# Remove empty geometries
counties_gdf = counties_gdf[~counties_gdf.geometry.is_empty]
if counties_gdf.empty:
    print("Error: No valid geometries after repair. Please check the data!")
    exit()

# Check if CRS is EPSG:4326
if counties_gdf.crs.to_epsg() != 4326:
    print("Warning: GeoJSON CRS is not EPSG:4326. Reprojecting...")
    counties_gdf = counties_gdf.to_crs(epsg=4326)

# Get list of all population TIF files
print("Retrieving list of TIF files...")
population_tifs = glob.glob(os.path.join(input_tif_directory, 'global_*_2020_1km.tif'))

if not population_tifs:
    print("No matching TIF files found. Please check the input directory.")
    exit()

# Print GeoJSON and TIF file bounds for debugging
print("GeoJSON bounds:", counties_gdf.total_bounds)


def clip_raster(tif_path, counties_gdf):
    """
    Clip a single TIF file using European region boundaries and save to the specified location.
    """
    output_filename = os.path.basename(tif_path).replace('.tif', '_clipped.tif')
    output_path = os.path.join(output_tif_directory, output_filename)
    
    try:
        print(f"Processing file: {tif_path}")
        with rasterio.open(tif_path) as src:
            # Check if the TIF file uses EPSG:4326
            if src.crs.to_epsg() != 4326:
                raise ValueError(f"File {tif_path} does not use EPSG:4326. Cannot process.")

            # Check if TIF and GeoJSON bounds intersect
            tif_bounds = src.bounds
            geo_bounds = counties_gdf.total_bounds
            if (tif_bounds[2] < geo_bounds[0] or  # TIF max longitude < GeoJSON min longitude
                tif_bounds[0] > geo_bounds[2] or  # TIF min longitude > GeoJSON max longitude
                tif_bounds[3] < geo_bounds[1] or  # TIF max latitude < GeoJSON min latitude
                tif_bounds[1] > geo_bounds[3]):   # TIF min latitude > GeoJSON max latitude
                print(f"Warning: File {tif_path} has no spatial overlap with GeoJSON. Skipping.")
                return

            # Clip the raster
            out_image, out_transform = mask(src, counties_gdf.geometry, crop=True)

            # Check if the clipped result is empty
            if out_image.size == 0:
                print(f"Warning: Clipping result for {tif_path} is empty. Skipping save.")
                return

            out_meta = src.meta.copy()
            
            # Update metadata to reflect the new extent
            out_meta.update({
                "driver": "GTiff",
                "height": out_image.shape[1],
                "width": out_image.shape[2],
                "transform": out_transform
            })
            
            # Write the clipped TIF file
            with rasterio.open(output_path, "w", **out_meta) as dest:
                dest.write(out_image)
                
            print(f"Successfully clipped and saved: {output_path}")
    except RasterioIOError as rio_err:
        print(f"Cannot open file {tif_path}: {rio_err}")
    except ValueError as ve:
        print(f"Problem with file {tif_path}: {ve}")
    except Exception as e:
        print(f"Error during clipping {tif_path}: {e}")


# Process each TIF file sequentially
print("Starting to clip TIF files...")
for tif in population_tifs:
    clip_raster(tif, counties_gdf)

print("All files have been clipped successfully!")

#   Multiband synthetic population in Europe

In [ ]:
import os
import glob
import rasterio
from rasterio.enums import Resampling

# Input and output path settings (adjust according to actual paths)
clipped_tif_directory = r'欧洲裁剪后人口1km全部'
output_multiband_tif_path = r'merged_population_data_EUnew.tif'

# Get all clipped population TIF files
clipped_population_tifs = glob.glob(os.path.join(clipped_tif_directory, '*_clipped.tif'))

# Ensure at least one file is available to create a multiband TIF
if not clipped_population_tifs:
    print("No clipped TIF files found.")
else:
    # Use the first file to get metadata template
    with rasterio.open(clipped_population_tifs[0]) as src:
        meta = src.meta.copy()
        meta.update(count=len(clipped_population_tifs))

    # Prepare list for band descriptions
    band_descriptions = []

    # Open a new file to write multiband data
    with rasterio.open(output_multiband_tif_path, 'w', **meta) as dst:
        for band_index, tif in enumerate(clipped_population_tifs, start=1):
            try:
                # Generate band description from filename
                filename = os.path.basename(tif)
                # Assume filename format: global_[f|m]_[age]_[year]_1km_clipped.tif or similar
                parts = filename.split('_')
                if len(parts) >= 4 and parts[-1].startswith('clipped'):
                    band_name = f"{parts[1]}_{parts[2]}"  # e.g., f_20 or m_75
                else:
                    band_name = f"band_{band_index}"  # default name

                print(f"Processing file {tif} as band {band_index} ({band_name})")

                # Write single-band data into the multiband file
                with rasterio.open(tif) as src:
                    data = src.read(1)
                    dst.write(data, band_index)

                # Add band description
                band_descriptions.append((band_index, band_name))
            except Exception as e:
                print(f"Error processing file {tif}: {e}")

        # Write band descriptions as tags in the multiband TIF file
        dst.update_tags(ns='rio_band_descriptions', **{str(i): desc for i, desc in band_descriptions})

    print(f"All files have been successfully merged into multiband TIF: {output_multiband_tif_path}")
    print("Band descriptions:")
    for i, desc in band_descriptions:
        print(f"Band {i}: {desc}")